In [59]:
from data.data import dataset_attributes, shift_types, prepare_data, log_data
from utils import set_seed, Logger, CSVBatchLogger, log_args, get_model, hinge_loss
from notebooks.example_args import MyCUBArgs
import torch 
from data import dro_dataset
import wandb, os
from train import train
from data.folds import Subset, ConcatDataset

mainargs = MyCUBArgs(wandb=False,
                     seed=0,
                     show_progress=False,
                     project_name='')  # default gpu = 0
part1_args = mainargs.part1_args
part2_args = mainargs.part2_args
args = part1_args
set_seed(args.seed)

train_data, val_data, test_data = prepare_data(args, train=True)


Reading '/home/thien/research/datasets/cub/data/waterbird_complete95_forest2water2/metadata.csv'


In [63]:
args

{'q': 0.7,
 'lr': 0.001,
 'btl': False,
 'gpu': 0,
 'fold': None,
 'seed': 0,
 'alpha': 0.2,
 'gamma': 0.1,
 'hinge': False,
 'model': 'resnet50',
 'wandb': False,
 'resume': False,
 'aug_col': 'None',
 'dataset': 'CUB',
 'log_dir': '/SPGL_proportion0.5_epochs51_lr0.001_weightdecay0.0001/part1',
 'fraction': 1,
 'n_epochs': 51,
 'root_dir': '/home/thien/research/datasets/cub',
 'run_test': False,
 'log_every': 151,
 'loss_type': 'erm',
 'save_best': False,
 'save_last': False,
 'save_step': 10,
 'scheduler': False,
 'batch_size': 32,
 'num_sweeps': 4,
 'shift_type': 'confounder',
 'target_name': 'waterbird_complete95',
 'augment_data': False,
 'project_name': '',
 'val_fraction': 0.1,
 'weight_decay': 0.0001,
 'subsample_minority': False,
 'multi_subsample': False,
 'metadata_path': 'myresults/CUB/waterbird_newrun/metadata_aug.csv',
 'show_progress': False,
 'imbalance_ratio': None,
 'joint_dro_alpha': 1,
 'reweight_groups': False,
 'use_bert_params': 1,
 'confounder_names': ['forest2w

In [61]:
dataset = val_data

indices = np.array([], dtype=int)
remaining_idxs = np.array([], dtype=int)
group_array = dataset.get_group_array()

print(f"[utils/get_group_balanced...] len(dataset)={len(dataset)}")

for g in np.arange(dataset.n_groups):
    group_indices = np.where((group_array == g))[0]
    num_g = int(math.ceil(len(group_indices) * split_fraction))
    print(f"[utils/get_group_balanced...] g={g}: len(group_indices)={len(group_indices)}")
    permuted_indices = np.sort(np.random.permutation(group_indices))
    indices = np.concatenate((indices, permuted_indices[:num_g]))
    remaining_idxs = np.concatenate((remaining_idxs, permuted_indices[num_g:]))

[utils/get_group_balanced...] len(dataset)=1199
[utils/get_group_balanced...] g=0: len(group_indices)=467
[utils/get_group_balanced...] g=1: len(group_indices)=466
[utils/get_group_balanced...] g=2: len(group_indices)=133
[utils/get_group_balanced...] g=3: len(group_indices)=133


In [62]:
dataset

In [54]:
data = val_data

split_fraction = .10
group_counts = data.group_counts()
sampled_group_counts = torch.ceil(group_counts*split_fraction)

indices = np.array([], dtype=int)
group_array = data.get_group_array()

for g in np.arange(data.n_groups):
    group_indices = np.where((group_array == g))[0]
    num_g = int(torch.ceil(group_counts[g]*split_fraction))
    print(f"group_count={group_counts[g]} vs. len={len(group_indices)}")
    indices = np.concatenate((
        indices, np.sort(np.random.permutation(group_indices)[:num_g])))


group_count=467.0 vs. len=467
group_count=466.0 vs. len=466
group_count=133.0 vs. len=133
group_count=133.0 vs. len=133


In [55]:
dataset = data

indices = np.array([], dtype=int)
remaining_idxs = np.array([], dtype=int)
group_array = dataset.get_group_array()
group_counts = data.group_counts()

for g in np.arange(dataset.n_groups):
    print(f"\ng={g}")
    group_indices = np.where((group_array == g))[0]
    print(f"group_count={group_counts[g]} vs. len={len(group_indices)}")
    num_g = int(math.ceil(len(group_indices) * split_fraction))
    print(f'num_g=', num_g)
    permuted_indices = np.sort(np.random.permutation(group_indices))
    print(f"permuted_indices_len={len(permuted_indices)}")
    indices = np.concatenate((indices, permuted_indices[:num_g]))
    remaining_idxs = np.concatenate((remaining_idxs, permuted_indices[num_g:]))
    print(f"indices={len(indices)} vs. remaining_idxs={len(remaining_idxs)}.")
    print(f"sum={len(indices)+len(remaining_idxs)}")


g=0
group_count=467.0 vs. len=467
num_g= 47
permuted_indices_len=467
indices=47 vs. remaining_idxs=420.
sum=467

g=1
group_count=466.0 vs. len=466
num_g= 47
permuted_indices_len=466
indices=94 vs. remaining_idxs=839.
sum=933

g=2
group_count=133.0 vs. len=133
num_g= 14
permuted_indices_len=133
indices=108 vs. remaining_idxs=958.
sum=1066

g=3
group_count=133.0 vs. len=133
num_g= 14
permuted_indices_len=133
indices=122 vs. remaining_idxs=1077.
sum=1199


In [58]:
len(indices)/len(dataset)

0.10175145954962468

In [39]:
from data.data import dataset_attributes, shift_types, prepare_data, log_data
from utils import set_seed, Logger, CSVBatchLogger, log_args, get_model, hinge_loss
from notebooks.example_args import MyCUBArgs
import torch 
from data import dro_dataset
import wandb, os
from train import train
from data.folds import Subset, ConcatDataset

mainargs = MyCUBArgs(wandb=False,
                     seed=0,
                     show_progress=False,
                     project_name='')  # default gpu = 0
part1_args = mainargs.part1_args
part2_args = mainargs.part2_args
args = part1_args
set_seed(args.seed)

train_data, val_data, test_data = prepare_data(args, train=True)


Reading '/home/thien/research/datasets/cub/data/waterbird_complete95_forest2water2/metadata.csv'


In [41]:
from my_run_expt import make_data_split

p1, p2 = make_data_split(val_data,)

tensor([467., 466., 133., 133.])